## [How does Python SK compare to the C# version of Semantic Kernel?](https://github.com/microsoft/semantic-kernel/blob/main/python/README.md)
- The two SDKs are compatible and at the core they follow the same design principles.
- Some features are still available only in the C# version, and being ported.
- Refer to the [FEATURE MATRIX](https://learn.microsoft.com/en-us/semantic-kernel/get-started/supported-languages) doc to see where things stand in matching the features and functionality of the main SK branch.
- Over time there will be some features available only in the Python version, and others only in the C# version, for example adapters to external services, scientific libraries, etc.
<br/>
- Documentation
 - [Get Started with Semantic Kernel](https://github.com/microsoft/semantic-kernel/blob/main/python/README.md)

Inspired to
 - https://github.com/microsoft/semantic-kernel/blob/main/dotnet/samples/KernelSyntaxExamples/Example59_OpenAIFunctionCalling.cs
 - https://github.com/microsoft/semantic-kernel/blob/main/python/samples/kernel-syntax-examples/openai_function_calling.py

In [1]:
# create the kernel

import semantic_kernel as sk
kernel = sk.Kernel()

In [3]:
# create the SK "chat completion connector" to the Azure OpenAI service

from dotenv import load_dotenv
load_dotenv("credentials_my.env")

import semantic_kernel.connectors.ai.open_ai as sk_oai

completion_connector = sk_oai.AzureChatCompletion(
    api_key          = os.environ['AZURE_OPENAI_API_KEY'],
    api_version      = os.environ['AZURE_OPENAI_API_VERSION'],
    deployment_name  = os.environ['GPT4-1106-128k'], # ['GPT4-1106-128k'],
    endpoint=os.environ['AZURE_OPENAI_ENDPOINT']
)
print(f"This is my AzureChatCompletion connector:\n{completion_connector}")

This is my AzureChatCompletion connector:
ai_model_id='gpt4-1106-128k' client=<openai.lib.azure.AsyncAzureOpenAI object at 0x7f3e48f1e540> ai_model_type=<OpenAIModelTypes.CHAT: 'chat'> prompt_tokens=0 completion_tokens=0 total_tokens=0


In [4]:
# add the openAI completion connector to the kernel, choosing a name to identify it within the kernel

kernel.add_chat_service(
    "mauromi_chatgpt", # unique name to be registered with the kernel
    completion_connector
)
print(f"Here are all chat services registered with this kernel: {kernel.all_chat_services()}")

Here are all chat services registered with this kernel: ['mauromi_chatgpt']


In [5]:
# Import a built-in plugin

from semantic_kernel.core_plugins import MathPlugin
kernel.import_plugin(MathPlugin(), plugin_name="mauromi_calculator")

print(f"Here are my plugins imported into the kernel:\n{kernel.plugins}")

Here are my plugins imported into the kernel:
plugins={'mauromi_calculator': KernelPlugin(name='mauromi_calculator', description=None, functions={'Add': KernelFunction(plugin_name='mauromi_calculator', description='Adds value to a value', name='Add', is_semantic=False, stream_function=<bound method MathPlugin.add of MathPlugin()>, parameters=[ParameterView(name='input', description='The value to add', default_value='', type_='string', required=False), ParameterView(name='Amount', description='Amount to add', default_value='', type_='number', required=True)], delegate_type=<DelegateTypes.InStringAndContextOutString: 12>, function=<bound method MathPlugin.add of MathPlugin()>, plugins=KernelPluginCollection(plugins={...}), ai_service=None, prompt_execution_settings=PromptExecutionSettings(service_id=None, extension_data={}), chat_prompt_template=None), 'Subtract': KernelFunction(plugin_name='mauromi_calculator', description='Subtracts value to a value', name='Subtract', is_semantic=False

In [ ]:
# Import custom semantic plugin
# the joke plugin in the FunPlugins is a semantic plugin and has the function calling disabled.

import os
plugins_directory = os.path.join(os.getcwd(), "./Plugins")
kernel.import_semantic_plugin_from_directory(plugins_directory, "FunPlugin")

print(f"Here are my plugins imported into the kernel:\n{kernel.plugins}")

In [ ]:
# custom function that can be used with Function Calling
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_hotels",
            "description": "Retrieves hotels from the search index based on the parameters provided",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location of the hotel (i.e. Seattle, WA)",
                    },
                    "max_price": {
                        "type": "number",
                        "description": "The maximum price for the hotel",
                    },
                    "features": {
                        "type": "string",
                        "description": "A comma separated list of features (i.e. beachfront, free wifi, etc.)",
                    },
                },
                "required": ["location"],
            },
        },
    }
]

Enabling or disabling function calling is done by setting the `function_call` parameter for the completion.<br/>
When the function_call parameter is set to "auto" the model will decide which function to use, if any.<br/>
If you only want to use a specific function, set the name of that function in this parameter, the format for that is 'PluginName-FunctionName', (i.e. 'math-Add').<br/>
If the model or api version do not support this you will get an error.<br/>

In [ ]:
# prompt template config that contains the definition and description of each function

prompt_config = sk.PromptTemplateConfig.from_execution_settings(
    max_tokens=2000,
    temperature=0.7,
    top_p=0.8,
    tool_choice="auto",
    tools=tools,
)
prompt_config

In [ ]:
kernel.get_text_completion_service_service_id()

In [ ]:
# now we bind the 

prompt_template = sk.ChatPromptTemplate[sk_oai.models.chat.open_ai_chat_message.OpenAIChatMessage](
    "{{$user_input}}", kernel.prompt_template_engine, prompt_config
)
prompt_template

In [ ]:
system_message = """
You are a chat bot. Your name is Mosscap and
you have one goal: figure out what people need.
Your full name, should you need to know it, is
Splendid Speckled Mosscap. You communicate
effectively, but you tend to answer with long
flowery prose. You are also a math wizard, 
especially for adding and subtracting.
You also excel at joke telling, where your tone is often sarcastic.
Once you have the answer I am looking for, 
you will return a full answer to me as soon as possible.
"""

In [ ]:
prompt_template.add_system_message(system_message)
prompt_template.add_user_message("Hi there, who are you?")
prompt_template.add_assistant_message("I am Mosscap, a chat bot. I'm trying to figure out what people need.")
prompt_template

In [ ]:
prompt_template.messages

In [ ]:
function_config = sk.SemanticFunctionConfig(prompt_template=prompt_template, prompt_template_config=prompt_config)
function_config

In [ ]:
function_config.prompt_template_config.execution_settings.extension_data["tools"]

In [ ]:
chat_function = kernel.register_semantic_function (
    plugin_name="ChatBot", 
    function_name="Chat", 
    function_config= function_config)

chat_function.describe()

In [ ]:
context = kernel.create_new_context()
context.variables["user_input"] = "I want to find a hotel in Seattle with free wifi and a pool."
response = chat_function.invoke(context=context)
response

In [ ]:
for message in response:
    print(f"message: {message}")

In [ ]:
messages = []
tool_call = None

for message in response:
    print(f"message: {message}")
    current = message[0]
    messages.append(current)
    if current.tool_calls:
        if tool_call is None:
            tool_call = current.tool_calls[0]
        else:
            tool_call += current.tool_calls[0]